# Intro to HTTP caching in python

In [ ]:
# Demo dependencies and helper functions
!pip install -U requests requests-cache rich pygithub pyinaturalist pymongo

from typing import Mapping
from rich import pretty, print


def pprint(obj):
    """A bit of extra formatting when printing a dictionary of headers"""
    if isinstance(obj, Mapping):
        for k in sorted(obj):
            print(f'[bold cyan]{k}[/bold cyan]: {obj[k]}')
    else:
        pretty.pprint(obj)

# Caching basics
Basic example with [functools.lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache):

In [ ]:
from functools import lru_cache
from datetime import datetime, timedelta
import requests


@lru_cache(maxsize=32)
def get_pep(num):
    """Retrieve text of a Python Enhancement Proposal"""
    url = f'https://peps.python.org/pep-{num:0>4}/'
    return requests.get(url).text


for n in [8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991, 8]:
    pep = get_pep(n)
    print(f'PEP {n:<4} text length: {len(pep)}')


get_pep.cache_info()

Limitations:
* Cache is stored only in memory and not persistent
* Only works for static resources
* No way to specify if/when to refresh cached pages
* requests are matched based only on function arguments

# HTTP Caching basics

![](images/browser-cache-basic.png)

## Browser cache layers
![](images/browser-cache-types.png)

## Types of caches
* Server-side
* Public (proxy/edge/CDN)
* Private (browser/other client)

## Reference
Main specs (wall of text warning):
* [RFC 2616](https://datatracker.ietf.org/doc/html/rfc2616#section-13)
* [RFC 7234](https://datatracker.ietf.org/doc/html/rfc7234)

Extensions:
* [RFC 5861](https://datatracker.ietf.org/doc/html/rfc5861)
* [RFC 8246](https://datatracker.ietf.org/doc/html/rfc8246)

More readable summaries:
* [MDN: Cache-Control](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Cache-Control#up-to-date_contents_always)
* [MDN: ETag](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/ETag) and [If-None-Match](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/If-None-Match)

# requests-cache
The [requests-cache](https://github.com/requests-cache/requests-cache) library gives you an easy drop-in replacement for [requests.Session](https://requests.readthedocs.io/en/latest/user/advanced/#session-objects):

In [ ]:
from requests import Session
from requests_cache import CachedSession


# Instead of this:
session = Session()
r = session.get('https://httpbin.org/delay/2')

# Do this:
session = CachedSession()
r = session.get('https://httpbin.org/delay/2')

## Benchmark
After being cached, the only I/O operation for a given request is reading from SQLite.

See [this example](https://requests-cache.readthedocs.io/en/stable/examples.html#cache-speed-test) for a more complete benchmark script with options

In [ ]:
from time import perf_counter


start = perf_counter()

for i in range(100):
    r = session.get('https://httpbin.org/delay/2')
    
elapsed = (perf_counter() - start) * 1000
print(f'Elapsed: {elapsed:.5}ms (avg {elapsed/100:.4}ms per request)')

# Expiration strategies
* No expiration (default)
* User-specified expiration
* Header-based expiration (`Cache-Control`, etc.)

In [ ]:
from datetime import timedelta
from requests_cache import DO_NOT_CACHE, NEVER_EXPIRE, EXPIRE_IMMEDIATELY, CachedSession

# Specify a simple expiration value in seconds
session = CachedSession(expire_after=60)

# To specify a unit of time other than seconds, use a timedelta
session = CachedSession(expire_after=timedelta(days=30))

# Update an existing session to store new responses indefinitely
session.settings.expire_after = NEVER_EXPIRE

# Disable caching by default, unless enabled by other settings
session = CachedSession(expire_after=DO_NOT_CACHE)

# Cache the response but mark it as stale; will be revalidated on next use
session = CachedSession(expire_after=EXPIRE_IMMEDIATELY)

# Set expiration with URL glob patterns
urls_expire_after = {
    '*.site_1.com': 30,
    'site_2.com/resource_1': 60 * 2,
    'site_2.com/resource_2': 60 * 60 * 24,
    'site_2.com/resource_*': 60 * 60,
    'site_2.com/static': NEVER_EXPIRE,
}
session = CachedSession(urls_expire_after=urls_expire_after)

# Use case: iNaturalist API

* [iNaturalist](https://www.inaturalist.org) is a community science platform for posting and identifying observations of living things.
* It's also useful as a biodiversity data source, which can be accessed via a [REST API](https://api.inaturalist.org/v1/docs).
* The API is rate-limited and a bit slow, so caching is extremely useful here

Example observation search for Swamp Milkweed in Iowa (web UI): https://www.inaturalist.org/observations?taxon_id=125381&place_id=24

Equivalent API request:

In [ ]:
session = CachedSession(cache_control=True)
# session.cache.clear()

r = session.get(
    'https://api.inaturalist.org/v1/observations',
    params={'taxon': 'Asclepias incarnata', 'place_id': 24},
)

In [ ]:
pprint(r)
pprint(r.headers)
# print(r.json())

TL;DR: I made a client library with caching by default: https://github.com/pyinat/pyinaturalist

# GitHub API example

[GitHub REST API docs](https://docs.github.com/en/rest)

* **Bad news:** It has [rate limiting](https://docs.github.com/en/rest/rate-limit), which is especially restrictive for unauthenticated requests
* **Good news:** It supports conditional requests, which don't count against the rate limit!

## Direct REST API usage

In [ ]:
r = session.get('https://api.github.com/repos/requests-cache/requests-cache')
pprint(r.headers)
print(r.json())

In [ ]:
# Expiration and current rate limit usage:
print(r.expires)
print(r.headers['X-RateLimit-Used'])

In [ ]:
# Manually override expiration time to trigger revalidation
expired_dt = datetime.utcnow() - timedelta(seconds=1)
session.cache.save_response(r, expires=expired_dt)

# A conditional request will be sent, which doesn't count against the rate limit
r = session.get('https://api.github.com/repos/requests-cache/requests-cache')
print(f'From cache: {r.from_cache}')
print(f'Revalidated: {r.revalidated}')
print(f'Expires: {r.expires}')
print(f"Rate limit used: {r.headers['X-RateLimit-Used']}")

## PyGithub usage
* [PyGithub](https://pygithub.readthedocs.io) is a full API client for GitHub, much nicer to use than plain API requests
* It uses `requests`, but its internal session isn't easily accessible
* In this case, we can use [install_cache()](https://requests-cache.readthedocs.io/en/stable/modules/requests_cache.patcher.html#requests_cache.patcher.install_cache) to  globally patch requests
    * ⚠️ See [caveats and warnings here](https://requests-cache.readthedocs.io/en/stable/user_guide/general.html#patching-limitations-potential-issues)!
    
See [full example script](https://requests-cache.readthedocs.io/en/stable/examples.html#pygithub)

In [ ]:
# Basic usage
from github import Github
from requests_cache import install_cache

install_cache(cache_control=True)

gh = Github()
user = gh.get_user('jwcook')
? user

# Cache contents

Examples of getting more info about the cache
* `session.cache` is a [BaseCache](https://requests-cache.readthedocs.io/en/stable/modules/requests_cache.backends.base.html#requests_cache.backends.base.BaseCache) instance with higher-level cache operations
    * One subclass per backend; in this case, it's [SQLiteCache](https://requests-cache.readthedocs.io/en/stable/modules/requests_cache.backends.sqlite.html#requests_cache.backends.sqlite.SQLiteCache)
* `session.cache.responses` is a dict-like interface that handles lower-level storage operations
* `session.cache.redirects` is a secondary cache used to optimize redirect chains

In [ ]:
# Check if a specific URL is cached
session.cache.contains(url='https://httpbin.org/get?foo=bar')

In [ ]:
r = session.get('https://httpbin.org/get?foo=bar')
session.cache.contains(url='https://httpbin.org/get?foo=bar')

In [ ]:
# See all URLs in the cache
session.cache.urls()

In [ ]:
# Get unexpired responses
for response in session.cache.filter(expired=False):
    pprint(response)

In [ ]:
pprint(session.cache.responses)

In [ ]:
# The main response cache is a dict-like object
for k, v in session.cache.responses.items():
    print(f'Cache key: {k}\n{v}')

# Backends

[Supported cache backends](https://requests-cache.readthedocs.io/en/stable/user_guide/backends.html):
* SQLite
* Redis
* MongoDB
    * Optionally with GridFS
* DynamoDB
* Filesystem
* Memory (basic non-persistent cache)

## MongoDB example
See details on [MongoDB cache backend](https://requests-cache.readthedocs.io/en/stable/user_guide/backends/mongodb.html)

In [ ]:
from requests_cache import CachedSession, MongoCache

# Connecting to another Docker container; default is localhost:27017
backend = MongoCache(host='mongo', port=27017)
session = CachedSession(backend=backend)

r = session.get('https://httpbin.org/json')
r = session.get('https://api.github.com/repos/requests-cache/requests-cache')
print(r.json())